In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import os
import time
import pickle

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_train = 'drive/MyDrive/ProyectoDataScience/train.csv'
path_test = 'drive/MyDrive/ProyectoDataScience/test.csv'
path_val = 'drive/MyDrive/ProyectoDataScience/val.csv'

train=pd.read_csv(path_train, delimiter = ',')
#test=pd.read_csv(path_test, delimiter = ',')

In [ ]:
train['Distancia_Total'] = train['DistanciaInicio2'] - train['DistanciaInicio1']

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(train['Distancia_Total'].values.reshape(-1,1), train['Diferencia'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
train=[]#liberamos memoria
test=pd.read_csv(path_test, delimiter = ',')

In [ ]:
test['Distancia_Total'] = test['DistanciaInicio2'] - test['DistanciaInicio1']

In [ ]:
predictions = rf.predict(test['Distancia_Total'].values.reshape(-1,1))

In [ ]:
errors = abs(predictions - test['Diferencia'])

In [ ]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'segundos')

Mean Absolute Error: 199.38 segundos


In [ ]:
test = []

In [ ]:
val=pd.read_csv(path_val, delimiter = ',')

In [ ]:
val['Distancia_Total'] = val['DistanciaInicio2'] - val['DistanciaInicio1']

In [ ]:
def eval_range(model, dataset, top=60, bottom=0):
  condition1 = dataset["Diferencia"] >= bottom
  condition2 = dataset["Diferencia"] < top
  total_cond = np.logical_and(condition1, condition2)
  filtered = dataset[total_cond]
  if filtered.shape[0] > 0:
    y_val = filtered["Diferencia"]
    X_val = filtered['Distancia_Total'].values.reshape(-1,1)
    y_pred = model.predict(X_val)
    #rms = mean_squared_error(y_val, y_pred, squared=True)
    mae = mean_absolute_error(y_val, y_pred)
    #r2 = r2_score(y_val, y_pred)
    return mae
  else:
    return None

In [ ]:
mae = eval_range(rf, val[['Distancia_Total', 'Diferencia']], top=1e20, bottom=0)
print("Errores globales")
#print("\tRMS: \t{}".format(rms))
print("\tMAE: \t{}".format(mae))
#print("\tR2: \t{}\n".format(r2))


ranges = [[0, 5*60], [5*60, 10*60], [10*60, 20*60], [20*60, 40*60], [40*60, 60*60]]
for bottom, top in ranges:
  mae= eval_range(rf, val[['Distancia_Total', 'Diferencia']], top=top, bottom=bottom)
  print("Errores para el rango {}min a {}min".format(bottom//60, top//60))
  #print("\tRMS: \t{}".format(rms))
  print("\tMAE: \t{}".format(mae))
  #print("\tR2: \t{}\n".format(r2))

Errores globales
	MAE: 	197.2840548104081
Errores para el rango 0min a 5min
	MAE: 	60.364205714280274
Errores para el rango 5min a 10min
	MAE: 	160.58971253863822
Errores para el rango 10min a 20min
	MAE: 	275.18863939048197
Errores para el rango 20min a 40min
	MAE: 	386.2820244200918
Errores para el rango 40min a 60min
	MAE: 	673.8775766116485
